## 📚 Part 1: Data Exploration

Let's start by exploring our dataset and understanding the user behavior patterns.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

### Load Dataset

In [ ]:
# Load the preprocessed data
train_df = pd.read_csv('../data/processed/train_ratings.csv')
test_df = pd.read_csv('../data/processed/test_ratings.csv')
movies_df = pd.read_csv('../data/processed/movies.csv')

print("📊 Dataset Statistics:")
print("=" * 60)
print(f"Training ratings: {len(train_df):,}")
print(f"Testing ratings:  {len(test_df):,}")
print(f"Total movies:     {len(movies_df):,}")
print(f"Total users:      {train_df['UserID'].nunique():,}")
print(f"Total ratings:    {len(train_df) + len(test_df):,}")
print(f"\nData split:       {len(train_df)/(len(train_df)+len(test_df))*100:.1f}% train / {len(test_df)/(len(train_df)+len(test_df))*100:.1f}% test")

### Rating Distribution Analysis

In [ ]:
# Analyze rating distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram of ratings
ax1 = axes[0]
rating_counts = train_df['Rating'].value_counts().sort_index()
ax1.bar(rating_counts.index, rating_counts.values, color='#3498db', alpha=0.7, edgecolor='black')
ax1.set_xlabel('Rating (stars)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Number of Ratings', fontsize=12, fontweight='bold')
ax1.set_title('Rating Distribution in Training Set', fontsize=14, fontweight='bold')
ax1.set_xticks([1, 2, 3, 4, 5])
ax1.grid(axis='y', alpha=0.3)

# Add percentages
total = len(train_df)
for rating, count in rating_counts.items():
    percentage = (count / total) * 100
    ax1.text(rating, count, f'{percentage:.1f}%', ha='center', va='bottom', fontweight='bold')

# Box plot
ax2 = axes[1]
ax2.boxplot(train_df['Rating'], vert=True, patch_artist=True,
            boxprops=dict(facecolor='#2ecc71', alpha=0.7),
            medianprops=dict(color='red', linewidth=2))
ax2.set_ylabel('Rating (stars)', fontsize=12, fontweight='bold')
ax2.set_title('Rating Distribution (Box Plot)', fontsize=14, fontweight='bold')
ax2.set_xticklabels(['All Ratings'])
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print("\n📈 Rating Statistics:")
print("=" * 60)
print(f"Average rating:  {train_df['Rating'].mean():.2f} ⭐")
print(f"Median rating:   {train_df['Rating'].median():.2f} ⭐")
print(f"Most common:     {train_df['Rating'].mode()[0]:.0f} ⭐")
print(f"Standard dev:    {train_df['Rating'].std():.2f}")
print(f"\n💡 Insight: Users tend to rate movies they like (average {train_df['Rating'].mean():.2f}/5)")

### User Behavior Analysis

In [ ]:
# User activity analysis
user_activity = train_df.groupby('UserID').size()

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# User activity distribution
ax1 = axes[0]
ax1.hist(user_activity, bins=50, color='#9b59b6', alpha=0.7, edgecolor='black')
ax1.set_xlabel('Number of Ratings per User', fontsize=12, fontweight='bold')
ax1.set_ylabel('Number of Users', fontsize=12, fontweight='bold')
ax1.set_title('User Activity Distribution', fontsize=14, fontweight='bold')
ax1.axvline(user_activity.median(), color='red', linestyle='--', linewidth=2, label=f'Median: {user_activity.median():.0f}')
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3)

# Top 20 most active users
ax2 = axes[1]
top_users = user_activity.nlargest(20)
ax2.barh(range(len(top_users)), top_users.values, color='#e74c3c', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Number of Ratings', fontsize=12, fontweight='bold')
ax2.set_ylabel('User Rank', fontsize=12, fontweight='bold')
ax2.set_title('Top 20 Most Active Users', fontsize=14, fontweight='bold')
ax2.invert_yaxis()
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n👥 User Activity Statistics:")
print("=" * 60)
print(f"Average ratings per user:  {user_activity.mean():.0f}")
print(f"Median ratings per user:   {user_activity.median():.0f}")
print(f"Most active user:          {user_activity.max():,} ratings")
print(f"Least active user:         {user_activity.min()} ratings")

### Movie Popularity Analysis

In [ ]:
# Movie popularity
movie_ratings = train_df.groupby('MovieID').agg({
    'Rating': ['count', 'mean']
}).reset_index()
movie_ratings.columns = ['MovieID', 'num_ratings', 'avg_rating']
movie_ratings = movie_ratings.merge(movies_df[['MovieID', 'Title']], on='MovieID')

# Top 15 most rated movies
top_movies = movie_ratings.nlargest(15, 'num_ratings')

fig, ax = plt.subplots(figsize=(12, 8))
y_pos = np.arange(len(top_movies))
ax.barh(y_pos, top_movies['num_ratings'], color='#f39c12', alpha=0.7, edgecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels([title[:40] + '...' if len(title) > 40 else title for title in top_movies['Title']], fontsize=10)
ax.set_xlabel('Number of Ratings', fontsize=12, fontweight='bold')
ax.set_title('Top 15 Most Rated Movies', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

# Add average rating labels
for i, (ratings, avg) in enumerate(zip(top_movies['num_ratings'], top_movies['avg_rating'])):
    ax.text(ratings, i, f'  {avg:.1f}⭐', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n🎬 Movie Statistics:")
print("=" * 60)
print(f"Average ratings per movie: {movie_ratings['num_ratings'].mean():.0f}")
print(f"Median ratings per movie:  {movie_ratings['num_ratings'].median():.0f}")
print(f"Most rated movie:          {movie_ratings['num_ratings'].max():,} ratings")

---
## 🧮 Part 2: Understanding Bias in Ratings

Before building our models, we need to understand that ratings have inherent biases:
- **User bias**: Some users rate generously, others are harsh critics
- **Movie bias**: Some movies are universally loved, others are disliked
- **Global mean**: The overall average rating across all users and movies

In [ ]:
# Load the original user-item matrix
matrix_original = pd.read_csv('../data/processed/user_item_matrix_original.csv', index_col=0)

print("📊 User-Item Matrix:")
print("=" * 60)
print(f"Shape: {matrix_original.shape[0]:,} users × {matrix_original.shape[1]:,} movies")
print(f"Total cells: {matrix_original.shape[0] * matrix_original.shape[1]:,}")
print(f"Known ratings: {(~matrix_original.isna()).sum().sum():,}")
print(f"Missing ratings: {matrix_original.isna().sum().sum():,}")
print(f"Density: {(~matrix_original.isna()).sum().sum() / (matrix_original.shape[0] * matrix_original.shape[1]) * 100:.2f}%")
print(f"\n💡 The matrix is very sparse - users only rate ~4% of all movies")

### Calculate Biases

In [ ]:
# Calculate global mean and biases
global_mean = matrix_original.values[~np.isnan(matrix_original.values)].mean()

# User biases (tendency to rate higher or lower than average)
user_means = matrix_original.mean(axis=1)
user_bias = user_means - global_mean

# Item biases (movie popularity)
item_means = matrix_original.mean(axis=0)
item_bias = item_means - global_mean

print("🎯 Bias Analysis:")
print("=" * 60)
print(f"Global average rating: {global_mean:.3f} ⭐\n")

print("👥 User Bias Examples:")
print("-" * 60)
# Show extreme users
generous_users = user_bias.nlargest(5)
harsh_users = user_bias.nsmallest(5)

print("\nMost Generous Raters (rate higher than average):")
for uid, bias in generous_users.items():
    print(f"  User {uid}: {bias:+.2f} stars above average")

print("\nHarshest Critics (rate lower than average):")
for uid, bias in harsh_users.items():
    print(f"  User {uid}: {bias:+.2f} stars below average")

print("\n🎬 Movie Bias Examples:")
print("-" * 60)
# Get movie titles
movie_lookup = {row['MovieID']: row['Title'] for _, row in movies_df.iterrows()}

# Most loved movies
loved_movies = item_bias.nlargest(5)
print("\nMost Universally Loved (rated higher than average):")
for mid, bias in loved_movies.items():
    title = movie_lookup.get(int(mid), f"Movie {mid}")
    print(f"  {title[:50]:50s}: {bias:+.2f} stars")

# Most disliked movies
disliked_movies = item_bias.nsmallest(5)
print("\nMost Universally Disliked (rated lower than average):")
for mid, bias in disliked_movies.items():
    title = movie_lookup.get(int(mid), f"Movie {mid}")
    print(f"  {title[:50]:50s}: {bias:+.2f} stars")

### Visualize Bias Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# User bias distribution
ax1 = axes[0]
ax1.hist(user_bias.dropna(), bins=50, color='#3498db', alpha=0.7, edgecolor='black')
ax1.axvline(0, color='red', linestyle='--', linewidth=2, label='No Bias')
ax1.set_xlabel('User Bias (stars)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Number of Users', fontsize=12, fontweight='bold')
ax1.set_title('User Rating Bias Distribution', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3)

# Movie bias distribution
ax2 = axes[1]
ax2.hist(item_bias.dropna(), bins=50, color='#2ecc71', alpha=0.7, edgecolor='black')
ax2.axvline(0, color='red', linestyle='--', linewidth=2, label='No Bias')
ax2.set_xlabel('Movie Bias (stars)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Number of Movies', fontsize=12, fontweight='bold')
ax2.set_title('Movie Popularity Bias Distribution', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight:")
print("Both distributions are centered around zero, but there's significant variation.")
print("Accounting for these biases is crucial for accurate predictions!")

### Before and After Bias Correction

In [ ]:
# Remove biases from the matrix
matrix_centered = matrix_original.copy()

for i in range(len(matrix_centered)):
    for j in range(len(matrix_centered.columns)):
        if not np.isnan(matrix_centered.iloc[i, j]):
            matrix_centered.iloc[i, j] = (matrix_centered.iloc[i, j] - global_mean - 
                                         user_bias.iloc[i] - item_bias.iloc[j])

# Visualize the transformation
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Before
user_sample = matrix_original.iloc[:200].values[~np.isnan(matrix_original.iloc[:200].values)]
axes[0].hist(user_sample, bins=30, color='#e74c3c', alpha=0.7, edgecolor='black')
axes[0].axvline(global_mean, color='blue', linestyle='--', linewidth=2, label=f'Average = {global_mean:.2f}')
axes[0].set_xlabel('Rating (stars)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[0].set_title('BEFORE: Original Ratings', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# After
centered_sample = matrix_centered.iloc[:200].values[~np.isnan(matrix_centered.iloc[:200].values)]
axes[1].hist(centered_sample, bins=30, color='#2ecc71', alpha=0.7, edgecolor='black')
axes[1].axvline(0, color='blue', linestyle='--', linewidth=2, label='Average = 0')
axes[1].set_xlabel('Centered Value', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[1].set_title('AFTER: Bias-Corrected (Centered) Ratings', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Transformation Complete!")
print(f"Original mean: {user_sample.mean():.3f}")
print(f"Centered mean: {centered_sample.mean():.6f} (≈ 0)")
print("\n💡 Now the data shows pure user preferences, not biased tendencies!")

---
## 🎯 Part 3: Model Training Results

We trained two matrix factorization models:
1. **SVD** - Singular Value Decomposition
2. **PMF with Bias** - Probabilistic Matrix Factorization with bias terms

In [ ]:
# Load model metrics
with open('../reports/model_metrics.json', 'r') as f:
    metrics = json.load(f)

print("📊 Model Performance Summary:")
print("=" * 60)
print(f"\n🔵 SVD Model:")
print(f"   RMSE: {metrics['SVD_RMSE']:.4f}")
print(f"   Status: {'✅ Excellent!' if metrics['SVD_passes_audit'] else '❌ Needs improvement'}")

print(f"\n🟢 PMF with Bias Model:")
print(f"   RMSE: {metrics['PMF_RMSE']:.4f}")
print(f"   Improvement over SVD: {metrics['improvement_%']:.2f}%")
print(f"   Status: ✅ Strong performance!")

print("\n💡 Lower RMSE means better predictions!")

### Model Comparison Visualization

In [ ]:
# Create comparison bar chart
models = ['SVD', 'PMF with Bias']
rmse_values = [metrics['SVD_RMSE'], metrics['PMF_RMSE']]
colors = ['#3498db', '#2ecc71']

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(models, rmse_values, color=colors, alpha=0.8, edgecolor='black', linewidth=2, width=0.5)

# Add value labels
for bar, value in zip(bars, rmse_values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.4f}',
            ha='center', va='bottom', fontsize=13, fontweight='bold')

# Add improvement annotation
ax.annotate('', xy=(1, rmse_values[1]), xytext=(0, rmse_values[0]),
            arrowprops=dict(arrowstyle='<->', color='red', lw=2))
ax.text(0.5, (rmse_values[0] + rmse_values[1])/2,
        f'{metrics["improvement_%"]:.1f}% improvement',
        ha='center', fontsize=12, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

ax.set_ylabel('RMSE (Root Mean Squared Error)', fontsize=12, fontweight='bold')
ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax.set_ylim(0.8, max(rmse_values) * 1.1)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### PMF Training Convergence

In [ ]:
from IPython.display import Image

print("📈 PMF Model Training Progress:\n")
print("The chart below shows how the model learned over time.")
print("- Blue line: Training error (how well it fits training data)")
print("- Red line: Test error (how well it generalizes to new data)")
print("- Green star: Best point where the model stopped (early stopping)\n")

Image(filename='../reports/pmf_convergence.png', width=900)

---
## 🎬 Part 4: Generating Recommendations

Now let's use our trained models to generate movie recommendations!

In [ ]:
# Import recommendation system
import sys
sys.path.append('..')
from utils.recommendation import RecommendationSystem

# Initialize
print("🔧 Initializing recommendation system...\n")
rec_system = RecommendationSystem()
print("\n✅ Ready to generate recommendations!")

### Example: Recommendations for User 42

In [ ]:
# Select a user
sample_user = 42

print(f"\n👤 Analyzing User {sample_user}...\n")

# Get user's top-rated movies
top_rated = rec_system.get_top_rated_movies(sample_user, top_n=10)

print(f"⭐ User {sample_user}'s Favorite Movies (from history):\n")
print(top_rated[['Rank', 'Title', 'Rating']].to_string(index=False))

# Stats
avg_rating = top_rated['Rating'].mean()
print(f"\n📊 User's average rating: {avg_rating:.2f} ⭐")
if avg_rating > 4.0:
    print("💡 This user tends to rate generously!")
elif avg_rating < 3.0:
    print("💡 This user is a tough critic!")
else:
    print("💡 This user has balanced rating behavior.")

### SVD Recommendations

In [ ]:
# Get SVD recommendations
svd_recs = rec_system.generate_recommendations(sample_user, model='svd', top_n=10)

print(f"\n🔵 SVD Model Recommendations for User {sample_user}:\n")
print(svd_recs[['Rank', 'Title', 'Genres', 'PredictedRating']].to_string(index=False))

### PMF Recommendations

In [ ]:
# Get PMF recommendations
pmf_recs = rec_system.generate_recommendations(sample_user, model='pmf', top_n=10)

print(f"\n🟢 PMF Model Recommendations for User {sample_user}:\n")
print(pmf_recs[['Rank', 'Title', 'Genres', 'PredictedRating']].to_string(index=False))

### Compare Models Side-by-Side

In [ ]:
# Visual comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# SVD plot
ax1 = axes[0]
svd_top5 = svd_recs.head(5)
y_pos = np.arange(len(svd_top5))
ax1.barh(y_pos, svd_top5['PredictedRating'], color='#3498db', alpha=0.8, edgecolor='black')
ax1.set_yticks(y_pos)
ax1.set_yticklabels([title[:35] + '...' if len(title) > 35 else title for title in svd_top5['Title']], fontsize=10)
ax1.set_xlabel('Predicted Rating', fontsize=11, fontweight='bold')
ax1.set_title(f'SVD Top 5 Recommendations (User {sample_user})', fontsize=13, fontweight='bold')
ax1.set_xlim(0, 5.5)
ax1.invert_yaxis()
ax1.grid(axis='x', alpha=0.3)

# PMF plot
ax2 = axes[1]
pmf_top5 = pmf_recs.head(5)
ax2.barh(y_pos, pmf_top5['PredictedRating'], color='#2ecc71', alpha=0.8, edgecolor='black')
ax2.set_yticks(y_pos)
ax2.set_yticklabels([title[:35] + '...' if len(title) > 35 else title for title in pmf_top5['Title']], fontsize=10)
ax2.set_xlabel('Predicted Rating', fontsize=11, fontweight='bold')
ax2.set_title(f'PMF Top 5 Recommendations (User {sample_user})', fontsize=13, fontweight='bold')
ax2.set_xlim(0, 5.5)
ax2.invert_yaxis()
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Overlap analysis
svd_movies = set(svd_recs.head(10)['MovieID'])
pmf_movies = set(pmf_recs.head(10)['MovieID'])
overlap = svd_movies & pmf_movies

print(f"\n🔄 Recommendation Overlap Analysis:")
print("=" * 60)
print(f"Common movies in both top 10: {len(overlap)}")
print(f"Unique to SVD: {len(svd_movies - pmf_movies)}")
print(f"Unique to PMF: {len(pmf_movies - svd_movies)}")
print(f"\n💡 The models find {'similar' if len(overlap) >= 5 else 'different'} recommendations for this user!")

---
## 📊 Part 5: Model Evaluation Visualizations

Let's examine how well our models perform through detailed visualizations.

### Predicted vs Actual Ratings

In [ ]:
from IPython.display import Image

print("📊 Prediction Accuracy Comparison:\n")
print("These scatter plots show how closely our predictions match actual user ratings.")
print("- Points closer to the red line = better predictions")
print("- RMSE value shows overall prediction error\n")

Image(filename='../reports/predicted_vs_actual.png', width=1000)

### Performance Metrics Comparison

In [ ]:
print("📈 Detailed RMSE Comparison:\n")
print("Bar chart showing model performance with target thresholds.\n")

Image(filename='../reports/rmse_comparison.png', width=800)

### Most Recommended Movies Across All Users

In [ ]:
print("🎬 Popular Recommendations Across All Users:\n")
print("These histograms show which movies each model recommends most frequently.")
print("This reveals the 'go-to' recommendations each model tends to make.\n")

Image(filename='../reports/top_recommendations.png', width=1000)

---
## 🎯 Part 6: Interactive Recommendations

Try getting recommendations for different users!

In [ ]:
def get_user_recommendations(user_id, model='pmf', top_n=10):
    """
    Get personalized movie recommendations for any user.
    
    Parameters:
    - user_id: User ID (1-6040)
    - model: 'svd' or 'pmf' (default: 'pmf')
    - top_n: Number of recommendations (default: 10)
    """
    print(f"\n{'='*70}")
    print(f"🎬 Recommendations for User {user_id} using {model.upper()} model")
    print(f"{'='*70}\n")
    
    # Get user's history
    top_rated = rec_system.get_top_rated_movies(user_id, top_n=5)
    if len(top_rated) > 0:
        print(f"⭐ User {user_id}'s Top 5 Rated Movies:\n")
        print(top_rated[['Title', 'Rating']].to_string(index=False))
        print(f"\n   Average: {top_rated['Rating'].mean():.2f} ⭐")
    
    # Get recommendations
    recs = rec_system.generate_recommendations(user_id, model=model, top_n=top_n)
    
    print(f"\n\n🎯 Top {top_n} Recommended Movies:\n")
    display_recs = recs[['Rank', 'Title', 'Genres', 'PredictedRating']].copy()
    display_recs['PredictedRating'] = display_recs['PredictedRating'].round(2)
    print(display_recs.to_string(index=False))
    
    return recs

# Example usage:
print("\n💡 Try different users! For example:")
print("   get_user_recommendations(100, model='pmf', top_n=10)")
print("   get_user_recommendations(500, model='svd', top_n=15)")
print("\n")

In [ ]:
# Try it yourself! Change the user_id to explore different users
user_recs = get_user_recommendations(user_id=100, model='pmf', top_n=15)

---
## 🎓 Part 7: Key Learnings & Insights

### What We Discovered

1. **Bias Correction is Critical**
   - Users have different rating tendencies (generous vs harsh)
   - Movies have inherent popularity differences
   - Removing these biases significantly improves predictions

2. **Early Stopping Prevents Overfitting**
   - Models can memorize training data if trained too long
   - Monitoring test error helps us stop at the optimal point
   - PMF stopped at epoch 55 out of 100 (saved 45% of training time!)

3. **Model Comparison Insights**
   - PMF with bias outperforms SVD by 5.05%
   - PMF learns iteratively, allowing for better bias modeling
   - Both models benefit from explicit bias terms

4. **Data Sparsity Matters**
   - Users only rate ~4% of all movies
   - Matrix factorization handles sparse data well
   - Collaborative filtering leverages patterns from other users

5. **Recommendation Diversity**
   - Different models recommend different movies
   - Ensemble approaches could combine both models
   - User preference patterns are complex and multi-faceted

---
## 🚀 Next Steps

### Try the Interactive Dashboard!

We've built a full Streamlit dashboard where you can:
- Get recommendations for any user (1-6040)
- Compare SVD vs PMF models side-by-side
- View user rating history and preferences
- Download recommendations as CSV files
- Explore all visualizations interactively

**To launch the dashboard:**
```bash
streamlit run app.py
```

### Explore Further:

1. **Try Different Users**: Each user has unique preferences - explore them!
2. **Experiment with Parameters**: Modify top_n to get more/fewer recommendations
3. **Compare Models**: See where SVD and PMF agree and disagree
4. **Analyze Genres**: Which genres does each model prefer?
5. **Build Features**: Use recommendations for hybrid models combining content and collaborative filtering

---

## 📚 Resources

- [MovieLens Dataset](https://grouplens.org/datasets/movielens/)
- [Matrix Factorization Techniques](https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf)
- [SVD for Recommendations](https://sifter.org/~simon/journal/20061211.html)
- [Probabilistic Matrix Factorization](https://papers.nips.cc/paper/2007/file/d7322ed717dedf1eb4e6e52a37ea7bcd-Paper.pdf)

---

<div style='text-align: center; padding: 2rem; background-color: #f8f9fa; border-radius: 10px; margin-top: 2rem;'>
    <h2>🎬 Thank you for exploring our Movie Recommender System! 🎬</h2>
    <p style='font-size: 1.2em; color: #555;'>Built with Matrix Factorization on MovieLens 1M Dataset</p>
    <p style='color: #777;'>SVD & PMF Models | 1M+ Ratings | 6K+ Users | 3.6K+ Movies</p>
</div>